In [ ]:
print("Hello World")

In [ ]:
## Load all libraries
import pandas as pd
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
import re
from bs4 import BeautifulSoup
import contractions
import csv
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# Load the dataset
file_path = "/root/workspace/aka_project/Naikdil/Datset_for_binary.csv"  # Update path as needed
df = pd.read_csv(file_path)

# Check and rename columns
print("Original columns:", df.columns)
df = df.rename(columns={'Base_Reviews': 'Review', 'Have_issue': 'Issue'})

# Convert 'Issue' to binary (1 for issue, 0 for no issue)
def convert_issue(issue):
    issue = str(issue).lower().strip()
    if issue == 'no' or issue == '0':
        return 0
    return 1  # Treat everything else as an issue

df['Issue'] = df['Issue'].apply(convert_issue)

# Check class distribution
class_counts = df['Issue'].value_counts()
print("\nClass Distribution:")
print(class_counts)

# Plot distribution of issues
plt.figure(figsize=(8, 4))
sns.countplot(data=df, x='Issue')
plt.title('Distribution of Issues (1=Issue, 0=No Issue)')
plt.xticks([0, 1], ['No Issue', 'Issue'])
plt.show()

### Functions for Preprocessing and Model Definition
def preprocess_review(text):
    # Ensure the text is a string before processing
    if not isinstance(text, str):
        text = str(text)  # Convert to string if it's not
    text = contractions.fix(text)
    text = re.sub(r'[^\w\s]', '', text)
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = ' '.join(text.split())
    return text

class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = int(self.labels[idx])
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

class DistilBERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(DistilBERTClassifier, self).__init__()
        self.bert = DistilBertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # Using the [CLS] token representation
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

def train(model, data_loader, optimizer, scheduler, device, class_weights=None):
    model.train()
    criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float).to(device))
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_loss = total_loss / len(data_loader)
    avg_accuracy = total_correct / total_samples
    return avg_loss, avg_accuracy

def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    total_loss = 0.0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
            loss = nn.CrossEntropyLoss()(outputs, labels)
            total_loss += loss.item()

    accuracy = accuracy_score(actual_labels, predictions)
    report = classification_report(actual_labels, predictions, output_dict=True)
    avg_loss = total_loss / len(data_loader)
    return accuracy, report, avg_loss, predictions, actual_labels

def predict_reviewtype(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)

    return "Issue" if preds.item() == 1 else "No Issue"

# Handle missing values
df['Review'] = df['Review'].fillna('')

# Generate sample from data
sampled_data = df.copy()
class_counts = sampled_data['Issue'].value_counts()
print("The distribution of sampled_data is:", class_counts)

# Apply preprocessing to the 'Review' column
sampled_data['ReviewP'] = sampled_data['Review'].apply(preprocess_review)

# Extract texts and labels
texts = sampled_data['ReviewP'].tolist()
labels = sampled_data['Issue'].tolist()

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Set up parameters
distbert_model_name = '/root/workspace/aka_project/Naikdil/distilbert_model'
num_classes = 2
max_length = 128
batch_size = 16
num_epochs = 6
learning_rate = 1e-5
tokenizer = DistilBertTokenizer.from_pretrained("/root/workspace/aka_project/Naikdil/distilbert_model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DistilBERTClassifier(distbert_model_name, num_classes).to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Prepare KFold cross-validation (5 splits)
kf = KFold(n_splits=5, random_state=42, shuffle=True)

train_losses = []
val_accuracies = []
val_reports = []
train_accuracies = []
val_losses = []

all_predictions = []
all_actual_labels = []

for train_index, val_index in kf.split(texts, labels):
    train_texts = np.array(texts)[train_index]
    train_labels = np.array(labels)[train_index]
    val_texts = np.array(texts)[val_index]
    val_labels = np.array(labels)[val_index]

    # Random oversampling
    ros = RandomOverSampler(random_state=42)
    train_texts_resampled, train_labels_resampled = ros.fit_resample(train_texts.reshape(-1, 1), train_labels)
    train_texts_resampled = train_texts_resampled.flatten()

    train_dataset = TextClassificationDataset(train_texts_resampled, train_labels_resampled, tokenizer, max_length)
    val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

    total_steps = len(train_dataloader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train_loss, train_accuracy = train(model, train_dataloader, optimizer, scheduler, device, class_weights)
        print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        val_accuracy, val_report, val_loss, predictions, actual_labels = evaluate(model, val_dataloader, device)
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
        print(val_report)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        val_reports.append(val_report)

        all_predictions.extend(predictions)
        all_actual_labels.extend(actual_labels)




In [ ]:
# Plot accuracy vs epoch
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Training Accuracy', color='blue')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy', color='red')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot loss vs epoch
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss', color='blue')
plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

# Confusion Matrix
cm = confusion_matrix(all_actual_labels, all_predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["No Issue", "Issue"], yticklabels=["No Issue", "Issue"])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()



In [ ]:
# ROC Curve
fpr, tpr, _ = roc_curve(all_actual_labels, all_predictions)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()



In [ ]:
# Calculate and print average metrics
avg_accuracy = np.mean([report['accuracy'] for report in val_reports])
macro_precision = np.mean([report['macro avg']['precision'] for report in val_reports])
macro_recall = np.mean([report['macro avg']['recall'] for report in val_reports])
macro_f1 = np.mean([report['macro avg']['f1-score'] for report in val_reports])
weighted_precision = np.mean([report['weighted avg']['precision'] for report in val_reports])
weighted_recall = np.mean([report['weighted avg']['recall'] for report in val_reports])
weighted_f1 = np.mean([report['weighted avg']['f1-score'] for report in val_reports])

print(f"Average Metrics:")
print(f"Average Accuracy: {avg_accuracy:.4f}")
print(f"Macro-Precision: {macro_precision:.4f}")
print(f"Macro-Recall: {macro_recall:.4f}")
print(f"Macro-F1 Score: {macro_f1:.4f}")
print(f"Weighted-Precision: {weighted_precision:.4f}")
print(f"Weighted-Recall: {weighted_recall:.4f}")
print(f"Weighted-F1 Score: {weighted_f1:.4f}")

In [ ]:
from sklearn.metrics import classification_report


# Generate classification report
report = classification_report(all_actual_labels, all_predictions, target_names=["No Issue", "Issue"])
print("Classification Report for DistilBERT Model:")
print(report)